# Neural Networks  For Classification Of Credit Card Transactions

#### Installing TensorFlow to build a neural network 

In [1]:
!pip install tensorflow

#### Importing Packages 

In [1]:
import tensorflow as tf

In [2]:
import pandas as pd 
import numpy as np

In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from subprocess import check_output

In [4]:
import pickle

In [6]:
from imblearn.over_sampling import SMOTE

In [8]:
from collections import Counter 

In [10]:
import tensorflow as tf

In [11]:
from tensorflow import keras

In [12]:
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import average_precision_score, confusion_matrix
import matplotlib.pyplot as plt

In [58]:
from sklearn import metrics

Reading and modifying the dataframe using SMOTE 

In [5]:
 df= pd.read_pickle('C:/Users/Vaishnavi M Shetty/Desktop/dfmod.pickle')

In [7]:
X = df.drop('Class', axis=1)
y = df['Class']

In [9]:
print('Original dataset shape %s' % Counter(y))
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_smote))
# Slit into train and test datasets
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote, y_smote, test_size=0.25, random_state=42)

Original dataset shape Counter({0: 284306, 1: 492})
Resampled dataset shape Counter({0: 284306, 1: 284306})


The dataframe has been processed and oversampled using SMOTE. Now, we will build an artificial neural network to train on our dataset <br>

## Building the model

In [17]:
model = Sequential()
model.add(Dense(64, activation="relu", input_dim=(X_train_smote.shape[1])))

model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(2, activation="relu"))

model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

Build a Sequential model using keras with 5 dense layers each with activation function "ReLu". The Output Layer uses sigmoid activation function to get a binary output.<br>
After much tuning we will use ADAM optimizer with a learning rate of 1e-4. <br>
We are not building a very deep neural network as the dataframe is not big.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2240      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 3

In [16]:
X_test_smote.drop('Time', axis=1, inplace=True)

E:\Users\Vaishnavi M Shetty\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
 X_train.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'average', 'min', 'time_difference', 'cond'],
      dtype='object')

In [48]:
X_train_smote.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'average',
       'min', 'time_difference', 'cond'],
      dtype='object')

## Training the Neural Network 

In [56]:
history = model.fit(X_train_smote, 
                    y_train_smote, 
                    epochs=100, 
                    validation_data=(X_test_smote, y_test_smote), 
                    callbacks=[ReduceLROnPlateau(patience=3, verbose=1, min_lr=1e-6), 
                               EarlyStopping(patience=5, verbose=1)])

Epoch 1/100
13327/13327 [==============================] - 17s 1ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 0.0094 - val_accuracy: 0.9972
Epoch 2/100
13327/13327 [==============================] - 16s 1ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0094 - val_accuracy: 0.9974
Epoch 3/100
13327/13327 [==============================] - 16s 1ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0094 - val_accuracy: 0.9972
Epoch 4/100
13327/13327 [==============================] - 17s 1ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0095 - val_accuracy: 0.9971
Epoch 5/100
13327/13327 [==============================] - 16s 1ms/step - loss: 0.0085 - accuracy: 0.9976 - val_loss: 0.0094 - val_accuracy: 0.9973
Epoch 6/100
13327/13327 [==============================] - 16s 1ms/step - loss: 0.0085 - accuracy: 0.9976 - val_loss: 0.0094 - val_accuracy: 0.9972
Epoch 7/100
13327/13327 [==============================] - 16s 1ms/step - loss: 0.0085 - accuracy: 0.9976 - val_

### Evaluating our model 

In [59]:
predictions = model.predict_classes(X_test_smote)
ap_score = metrics.roc_auc_score(y_test_smote, predictions)
print("The model test auc score is {}.".format(ap_score))

The model test auc score is 0.9975920125319466.


Saving the model weights in .h5 format using keras.

In [69]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_nn.h5")

In [70]:
file = open('model.json', 'r')
loaded  = file.read()
file.close()

loaded_model = model_from_json(loaded)
loaded_model.load_weights("model_nn.h5")

In [71]:
loaded_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                2176      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

THANK YOU